## Virtual Environment for Individual-Based Modeling

This virtual environment is based on an Agent-Based Modeling system using some assumptions that are derived from previousobservations of waterbirds’ behaviors within some habitats in the tropics.

Created on March 31, 2019 by Ralph Florent <r.florent@jacobs-university.de>

In [10]:
# Virtual Environment for Individual-Based Modeling (IBM)
#
# Created on March 31, 2019
#
# Authors:
#   Ralph Florent <r.florent@jacobs-university.de>
#   Davi Tavares <davi.tavares@leibniz-zmt.de>
#   Agostino Merico <a.merico@jacobs-university.de>
#
# Main entry point for the application

# ==============================================================================
# START: Preamble
# ==============================================================================

# -*- coding: utf-8 -*-
import config
import constants as CONST
from core import initialize, observe, update
from helpers import plot_figure, make_gif

# see http://stackoverflow.com/questions/1907993/autoreload-of-modules-in-ipython
# reload external python modules
%load_ext autoreload
%autoreload 2
%reload_ext autoreload


# ==============================================================================
# END: Preamble
# ==============================================================================

# ==============================================================================
# START: Application
# ==============================================================================

# main entry point for the application
# TODO: proper docs
def application():
    
    # pre-conditions
    config.init() # initialize internal config for the app
    time = 0 # define stopwatch for the process
    
     # process for t times
    print('=> START: Processing random movements based on prob dist')
    habitats, agents = initialize()
    observe(habitats, agents, time)
    
    for time in range(1, CONST.PROCESSING_TIME):
        agents = update(habitats, agents, time) # override agents when being updated
        observe(habitats, agents, time)
    
    print('=> END: Processing random movements based on prob dist')
    
    # post-conditions
    make_gif(CONST.ROOT_DIR, 'image.gif', CONST.STORE['images'])
    plot_figure()

    
# run application
application()

# ==============================================================================
# END: Application
# ==============================================================================

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
=> START: Processing random movements based on prob dist
--- process update: 1
-- habitat: orange-sm
---- short: s:0.40040, w:0.01136, d:0.00128, f:0.49972
-- habitat: orange-lg
---- long: s:0.00620, w:0.00869, d:0.00631, f:0.50870
-- habitat: orange-lg
---- short: s:0.40040, w:0.01136, d:0.00155, f:0.53659
-- habitat: green
---- short: s:0.04290, w:0.01104, d:0.00137, f:1.36084
-- habitat: orange-lg
---- short: s:0.40040, w:0.01136, d:0.00155, f:0.53659
-- habitat: blue
---- short: s:0.00840, w:0.01052, d:0.00349, f:0.75770
-- habitat: orange-lg
---- long: s:0.00620, w:0.00869, d:0.00631, f:0.50870
-- habitat: blue
---- short: s:0.00840, w:0.01052, d:0.00349, f:0.75770
-- habitat: green
---- short: s:0.04290, w:0.01104, d:0.00137, f:1.36084
-- habitat: green
---- short: s:0.04290, w:0.01104, d:0.00137, f:1.36084
-- habitat: orange-lg
---- long: s:0.00620, w:0.00869, d:0.00631, f:0.50870
-- habitat:

-- habitat: orange-lg
---- short: s:0.40040, w:0.01136, d:0.00155, f:0.53659
-- habitat: green
---- short: s:0.04290, w:0.01104, d:0.00137, f:1.36084
-- habitat: orange-lg
---- short: s:0.40040, w:0.01136, d:0.00155, f:0.53659
-- habitat: orange-sm
---- long: s:0.00620, w:0.00869, d:0.00632, f:0.50102
-- habitat: orange-lg
---- long: s:0.00620, w:0.00869, d:0.00631, f:0.50870
-- habitat: orange-lg
---- long: s:0.00620, w:0.00869, d:0.00631, f:0.50870
-- habitat: orange-sm
---- long: s:0.00620, w:0.00869, d:0.00632, f:0.50102
-- habitat: orange-sm
---- long: s:0.00620, w:0.00869, d:0.00632, f:0.50102
-- habitat: blue
---- short: s:0.00840, w:0.01052, d:0.00349, f:0.75770
-- habitat: orange-sm
---- long: s:0.00620, w:0.00869, d:0.00632, f:0.50102
-- habitat: orange-sm
---- long: s:0.00620, w:0.00869, d:0.00632, f:0.50102
-- habitat: orange-sm
---- long: s:0.00620, w:0.00869, d:0.00632, f:0.50102
-- habitat: orange-lg
---- long: s:0.00620, w:0.00869, d:0.00631, f:0.50870
-- habitat: orang

-- habitat: orange-lg
---- short: s:0.40040, w:0.00128, d:0.00155, f:0.53659
-- habitat: orange-lg
---- short: s:0.40040, w:0.00128, d:0.00155, f:0.53659
-- habitat: orange-sm
---- long: s:0.00620, w:0.00209, d:0.00632, f:0.50102
-- habitat: orange-lg
---- long: s:0.00620, w:0.00214, d:0.00631, f:0.50870
-- habitat: orange-sm
---- long: s:0.00620, w:0.00209, d:0.00632, f:0.50102
-- habitat: green
---- short: s:0.04290, w:0.14626, d:0.00137, f:1.36084
-- habitat: blue
---- short: s:0.00840, w:0.01198, d:0.00349, f:0.75770
-- habitat: blue
---- short: s:0.00840, w:0.01198, d:0.00349, f:0.75770
-- habitat: green
---- short: s:0.04290, w:0.14626, d:0.00137, f:1.36084
-- habitat: orange-lg
---- long: s:0.00620, w:0.00214, d:0.00631, f:0.50870
-- habitat: orange-lg
---- long: s:0.00620, w:0.00214, d:0.00631, f:0.50870
-- habitat: blue
---- short: s:0.00840, w:0.01198, d:0.00349, f:0.75770
-- habitat: orange-lg
---- long: s:0.00620, w:0.00214, d:0.00631, f:0.50870
-- habitat: green
---- short

-- habitat: orange-lg
---- long: s:0.00620, w:-0.00010, d:0.00631, f:0.50870
-- habitat: orange-lg
---- long: s:0.00620, w:-0.00010, d:0.00631, f:0.50870
-- habitat: orange-sm
---- short: s:0.40040, w:0.00829, d:0.00128, f:0.49972
-- habitat: orange-sm
---- long: s:0.00620, w:-0.00367, d:0.00632, f:0.50102
-- habitat: orange-lg
---- long: s:0.00620, w:-0.00010, d:0.00631, f:0.50870
-- habitat: orange-sm
---- short: s:0.40040, w:0.00829, d:0.00128, f:0.49972
-- habitat: orange-lg
---- long: s:0.00620, w:-0.00010, d:0.00631, f:0.50870
-- habitat: orange-lg
---- long: s:0.00620, w:-0.00010, d:0.00631, f:0.50870
-- habitat: orange-sm
---- short: s:0.40040, w:0.00829, d:0.00128, f:0.49972
-- habitat: orange-sm
---- short: s:0.40040, w:0.00829, d:0.00128, f:0.49972
-- habitat: orange-lg
---- long: s:0.00620, w:-0.00010, d:0.00631, f:0.50870
-- habitat: orange-lg
---- short: s:0.40040, w:0.00221, d:0.00155, f:0.53659
-- habitat: blue
---- short: s:0.00840, w:0.03165, d:0.00349, f:0.75770
-- h

-- habitat: orange-sm
---- long: s:0.00620, w:-0.00367, d:0.00632, f:0.50102
-- habitat: orange-lg
---- short: s:0.40040, w:0.00221, d:0.00155, f:0.53659
-- habitat: orange-lg
---- short: s:0.40040, w:0.00221, d:0.00155, f:0.53659
-- habitat: blue
---- short: s:0.00840, w:0.03165, d:0.00349, f:0.75770
-- habitat: orange-lg
---- short: s:0.40040, w:0.00221, d:0.00155, f:0.53659
-- habitat: orange-sm
---- long: s:0.00620, w:-0.00367, d:0.00632, f:0.50102
-- habitat: orange-sm
---- long: s:0.00620, w:-0.00367, d:0.00632, f:0.50102
-- habitat: orange-sm
---- long: s:0.00620, w:-0.00367, d:0.00632, f:0.50102
-- habitat: orange-lg
---- long: s:0.00620, w:-0.00010, d:0.00631, f:0.50870
-- habitat: orange-lg
---- short: s:0.40040, w:0.00221, d:0.00155, f:0.53659
-- habitat: orange-lg
---- short: s:0.40040, w:0.00221, d:0.00155, f:0.53659
-- habitat: orange-lg
---- long: s:0.00620, w:-0.00010, d:0.00631, f:0.50870
-- habitat: orange-lg
---- short: s:0.40040, w:0.00221, d:0.00155, f:0.53659
-- h

-- habitat: orange-lg
---- short: s:0.40040, w:0.00493, d:0.00155, f:0.53659
-- habitat: orange-sm
---- long: s:0.00620, w:0.00512, d:0.00632, f:0.50102
-- habitat: orange-sm
---- long: s:0.00620, w:0.00512, d:0.00632, f:0.50102
-- habitat: orange-lg
---- long: s:0.00620, w:0.00600, d:0.00631, f:0.50870
-- habitat: orange-lg
---- short: s:0.40040, w:0.00493, d:0.00155, f:0.53659
-- habitat: orange-lg
---- short: s:0.40040, w:0.00493, d:0.00155, f:0.53659
-- habitat: blue
---- short: s:0.00840, w:0.00492, d:0.00349, f:0.75770
-- habitat: orange-sm
---- long: s:0.00620, w:0.00512, d:0.00632, f:0.50102
-- habitat: blue
---- short: s:0.00840, w:0.00492, d:0.00349, f:0.75770
-- habitat: blue
---- short: s:0.00840, w:0.00492, d:0.00349, f:0.75770
-- habitat: orange-sm
---- long: s:0.00620, w:0.00512, d:0.00632, f:0.50102
-- habitat: orange-lg
---- long: s:0.00620, w:0.00600, d:0.00631, f:0.50870
-- habitat: green
---- short: s:0.04290, w:0.11902, d:0.00137, f:1.36084
-- habitat: orange-lg
--

-- habitat: orange-sm
---- long: s:0.00620, w:0.00512, d:0.00632, f:0.50102
-- habitat: orange-sm
---- short: s:0.40040, w:0.00351, d:0.00128, f:0.49972
-- habitat: orange-sm
---- long: s:0.00620, w:0.00512, d:0.00632, f:0.50102
-- habitat: orange-sm
---- short: s:0.40040, w:0.00351, d:0.00128, f:0.49972
-- habitat: blue
---- short: s:0.00840, w:0.00492, d:0.00349, f:0.75770
-- habitat: green
---- short: s:0.04290, w:0.11902, d:0.00137, f:1.36084
-- habitat: blue
---- short: s:0.00840, w:0.00492, d:0.00349, f:0.75770
-- habitat: green
---- short: s:0.04290, w:0.11902, d:0.00137, f:1.36084
-- habitat: green
---- short: s:0.04290, w:0.11902, d:0.00137, f:1.36084
-- habitat: orange-lg
---- long: s:0.00620, w:0.00600, d:0.00631, f:0.50870
-- habitat: blue
---- short: s:0.00840, w:0.00492, d:0.00349, f:0.75770
-- habitat: orange-lg
---- long: s:0.00620, w:0.00600, d:0.00631, f:0.50870
-- habitat: green
---- short: s:0.04290, w:0.11902, d:0.00137, f:1.36084
-- habitat: orange-lg
---- long: s

-- habitat: orange-lg
---- long: s:0.00620, w:0.00686, d:0.00631, f:0.50870
-- habitat: blue
---- short: s:0.00840, w:0.00397, d:0.00349, f:0.75770
-- habitat: orange-lg
---- short: s:0.40040, w:0.00664, d:0.00155, f:0.53659
-- habitat: green
---- short: s:0.04290, w:0.11360, d:0.00137, f:1.36084
-- habitat: orange-lg
---- short: s:0.40040, w:0.00664, d:0.00155, f:0.53659
-- habitat: orange-lg
---- long: s:0.00620, w:0.00686, d:0.00631, f:0.50870
-- habitat: orange-lg
---- long: s:0.00620, w:0.00686, d:0.00631, f:0.50870
-- habitat: orange-lg
---- long: s:0.00620, w:0.00686, d:0.00631, f:0.50870
-- habitat: green
---- short: s:0.04290, w:0.11360, d:0.00137, f:1.36084
-- habitat: orange-lg
---- long: s:0.00620, w:0.00686, d:0.00631, f:0.50870
-- habitat: orange-lg
---- short: s:0.40040, w:0.00664, d:0.00155, f:0.53659
-- habitat: orange-sm
---- long: s:0.00620, w:0.00569, d:0.00632, f:0.50102
-- habitat: orange-lg
---- long: s:0.00620, w:0.00686, d:0.00631, f:0.50870
-- habitat: orange-

=> END: Processing random movements based on prob dist


In [9]:
import os

os.getcwd()

'/home/rflorent/Perso/Storage/Education/JU/Semester II/Advanced Project I/IBM/src/notebooks'